In [1]:
import requests
from bs4 import BeautifulSoup
import json

lawsUrl = "https://indiankanoon.org/browselaws/"
response_lawsUrl = requests.get(lawsUrl)

soup_browse_laws = BeautifulSoup(response_lawsUrl.content, 'html.parser')
browse_laws_urls = []
browse_laws_hrefs = soup_browse_laws.select('div[class="info_indian_kanoon"] td a')
for each_laws_href in browse_laws_hrefs:
    browse_laws_urls.append("https://indiankanoon.org"+each_laws_href['href'])
print(browse_laws_urls)

india_laws = browse_laws_urls[0:4]
state_laws = browse_laws_urls[4:43]
british_laws = browse_laws_urls[43:]
print(india_laws)

['https://indiankanoon.org/browse/union-act/', 'https://indiankanoon.org/browse/constitution-amendments/', 'https://indiankanoon.org/browse/un-convention/', 'https://indiankanoon.org/browse/treaty-act/', 'https://indiankanoon.org/browse/andhra-act/', 'https://indiankanoon.org/browse/arunachal-act/', 'https://indiankanoon.org/browse/assam-act/', 'https://indiankanoon.org/browse/bihar-act/', 'https://indiankanoon.org/browse/chandigarh-act/', 'https://indiankanoon.org/browse/chattisgarh-act/', 'https://indiankanoon.org/browse/delhi-act/', 'https://indiankanoon.org/browse/goa-act/', 'https://indiankanoon.org/browse/gujarat-act/', 'https://indiankanoon.org/browse/haryana-act/', 'https://indiankanoon.org/browse/himachal-act/', 'https://indiankanoon.org/browse/jk-act/', 'https://indiankanoon.org/browse/jharkhand-act/', 'https://indiankanoon.org/browse/karnataka-act/', 'https://indiankanoon.org/browse/kerala-act/', 'https://indiankanoon.org/browse/madhyabharat-act/', 'https://indiankanoon.org/

In [ ]:
from pathlib import Path
import re

for each_law in india_laws:
    # print(each_law)
    response_each_law = requests.get(each_law)
    soup_each_law = BeautifulSoup(response_each_law.content, 'html.parser')
    law_urls = soup_each_law.select('div[class="info_indian_kanoon"] td a')
    for each_law_url in law_urls:
        folder = each_law_url['href'].split('/')[2]
        file_path = "./india_laws/"+folder+"/"
        url = "https://indiankanoon.org"+each_law_url['href']
        print(url)
        print(file_path)
        response_all_year = requests.get(url)


        soup = BeautifulSoup(response_all_year.content, 'html.parser')
        # Get the "entire year" link
        entire_year_link = soup.select('div[class="info_indian_kanoon"] tr:nth-of-type(1) td a')[0]
        response_all_year_first_page = requests.get('https://indiankanoon.org' + entire_year_link["href"])

        result_links = set()
        soup_all_year_first_page = BeautifulSoup(response_all_year_first_page.content, 'html.parser')
        all_year_links_first_page = soup_all_year_first_page.select('div[class="result"] > div[class="result_title"] > a')
        for each_link_in_first_page in all_year_links_first_page:
            result_links.add(f"https://indiankanoon.org{each_link_in_first_page["href"]}")

        # getting the other pages if pagination exists
        all_year_links_other_pages = soup_all_year_first_page.select('div[class="bottom"] > span[class="pagenum"] > a')
        for link in all_year_links_other_pages:
            each_page_url = "https://indiankanoon.org"+link['href']
            print(each_page_url)
            response_all_year_other_page = requests.get(each_page_url)
            soup_all_year_other_page = BeautifulSoup(response_all_year_other_page.content, 'html.parser')
            all_year_links_other_page = soup_all_year_other_page.select('div[class="result"] > div[class="result_title"] > a')
            for each_link_in_other_page in all_year_links_other_page:
                result_links.add(f"https://indiankanoon.org{each_link_in_other_page["href"]}")
        print(result_links)


        for i in result_links:
            response = requests.get(i)
            soup = BeautifulSoup(response.content, 'html.parser')
            hidden_tags = soup.find_all("input", type="hidden", value="pdf")
            a = hidden_tags[0].parent
            ac = a.get("action")
            pdfResponse = requests.post("https://indiankanoon.org" + ac, headers={}, data = {'type': 'pdf'})

            d = pdfResponse.headers['content-disposition']
            fname = re.findall("filename=(.+)", d)[0].replace('"','')
            print(fname)
            filename = Path(f'./pdfs/{fname}')
            filename.write_bytes(pdfResponse.content)